<img src="../Img/banner-fa-49.png">

Учебные материалы дисциплины "Программирование на языках Python и SQL" предназначены для семинарских занятий со студентами II курса Финансового университа при Правительстве Российской Федерации.

Автор: Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

Москва - 2021

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- Астахова И.Ф., Мельников В.М., Толстобров А.П., Фертиков В.В. СУБД: язык SQL в примерах и задачах.—М.:ФИЗМАТЛИТ, 2009. — 168 с. — ISBN 978-5-9221-0816-4.

В ряде учебных примеров использованы данные <a href="http://insideairbnb.com/get-the-data.html">Inside Airbnb</a>

<a id=T_3></a>
# Тема 3. SQLAlchemy ORM

<a id=Ref></a>
# Оглавление

[Введение](#Intro)<br>
[3.1. Определение таблиц через классы ORM](#T_3_1)<br>
[3.2. Сеанс](#T_3_2)<br>
[3.3. Вставка данных](#T_3_3)<br>
[3.4. Запросы](#T_3_4)<br>
[3.5. Отображение](#T_3_5)<br>
[3.6. Сеанс более подробно](#T_3_6)<br>
[3.7. Исключения](#T_3_7)<br>
[Задания](#T_3_8)<br>


<img src="../Img/Label_02.png">

Семинар № 9

16 апреля 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

17 апреля 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

23 апреля 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

<a id=Intro></a>
# Введение
[<= ](#Ref)||[ К оглавлению ](#Ref)||[ =>](#T_3_1)

ORM - Object-relational mapping - Объектно-реляционное отображение

SQLAlchemy ORM обеспечивает эффективный способ привязки схемы и операций базы данных к объектам данных.

В SQLAlchemy Core мы создавали контейнер метаданных, а затем объявляли объект *Table*, связанный с этими метаданными. В SQLAlchemy ORM мы будем определять класс, который наследуется от специального базового класса *declarative_base*. Этот базовый класс объединяет контейнер метаданных и средство сопоставления, которое сопоставляет наш класс с таблицей базы данных. Он также сопоставляет экземпляры класса с записями в этой таблице.

<a id=T_3_1></a>
[<= ](#Intro)||[ К оглавлению ](#Ref)||[ =>](#T_3_2)

# 3.1. Определение таблиц через классы ORM

ORM классы должны:
- Происходить от класса *declarative_base*.
- Содержать `__tablename__`, которое является именем таблицы базы данных.
- Содержать один или несколько атрибутов, которые являются объектами *Column*.
- Содержать атрибуты, составляющие первичный ключ.

Изучим требование, связанное с атрибутами. Определение столбцов в классе ORM похоже на определение столбцов в объекте *Table*, которое мы изучили в теме SQLAlchemy Core. Однако есть важное отличие. При определении столбцов в классе ORM в качестве имени столбца будет установлено имя атрибута класса, которому он назначен. Все остальное, что связано с типами данных и столбцами, применимо и здесь.

<img src="./Img/Listings_ORM_Schema.png">

<br><br>
Определим таблицу *listings* как класс ORM

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from datetime import datetime
from sqlalchemy import DateTime

Base = declarative_base()

In [ ]:
class Listing(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    

В этом примере *Base* - экземпляр класса *declarative_base()*. Затем создается дочерний класс *Listings*. Определяется имя таблицы 'listings'. Определяются атрибуты, устанавливается первичный ключ. Обратимся к свойству `__table__` класса.

In [ ]:
Listing.__table__

Создадим класс для клиентов

In [ ]:
from datetime import datetime
from sqlalchemy import DateTime

class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

Здесь мы определили несколько атрибутов, которые не могут оставаться пустыми. Требуется уникальное значение *username*. Для атрибута *updated_on* мы установили текущее время по умолчанию, если время не указано. Использование *onupdate* приведет к установке текущего времени при обновлении любого атрибута записи.

## 3.1.1. Ключи, ограничения, индексы

Ранее, в разделе *Core*, мы изучили, что ключи и ограничения могут задаваться как в составе элемента *Column()* конструктора *Table*, так и в явном виде. Например, в *line_items* атрибут *order_id* является внешним ключом, тода

`ForeignKeyConstraint(['order_id'], ['order.order_id'])`. 

В ORM также существует для этого два способа, но так как конструктор *Table* здесь не используется, то применяются свойства класса. 

```
user_id = Column(Integer(), ForeignKey('users.user_id'))
```

Для задания ограничения в явном виде в классе используется `__table_args__`

```
class SomeDataClass(Base):
    __tablename__ = 'somedatatable'
    __table_args__ = (ForeignKeyConstraint(['id'], ['other_table.id']),
                      CheckConstraint(price >= 0.00', name='unit_cost_positive'))
```
В данном примере значением `__table_args__` является кортеж.

### Задание 3.1.1.1.
Создать классы *Order, Line_item, Host, Neighbourhood, Room_type, Property_type*. Создать базу данных *Listings.db*

In [ ]:
# Ваш код здесь


In [ ]:
class Order(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)

In [ ]:
Order.__table__

In [ ]:
class Line_item(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)

In [ ]:
Line_item.__table__

In [ ]:
class Host(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)

In [ ]:
class Neighbourhood(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)

In [ ]:
class Room_type(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)

In [ ]:
class Property_type(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)

## 3.1.2. Сохранение схемы

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:') # a) В памяти
#engine = create_engine('sqlite:///Listings.db') # b) На диске

Base.metadata.create_all(engine)

## 3.1.3. Связи
В *ORM* имеются некоторые различия при связывании таблиц по сравнению с *Core*. *ORM* также использует *ForeignKey* для ограничения и связывания объектов. Однако *ORM* использует директиву *relationship* чтобы предоставить свойство доступа к связанному объекту. Это добавляет некоторые накладные расходы при использовании *ORM*; однако плюсы перевешивают недостатки. В примере показано, как определить связи с помощью методов *relationship* и *backref*.
```
from sqlalchemy.orm import relationship, backref

class Orders(Base):
    __tablename__='orders'
    order_id=Column(Integer(), primary_key=True)
    user_id=Column(Integer(), ForeignKey('users.user_id'))

    User=relationship('Users', backref=backref('orders', order_by=order_id))
```
Таким образом, в классе *Orders*, устанавливается отношение «один ко многим» с классом *Users*. Мы можем связать пользователя с его заказом, обратившись к свойству *user*. Это отношение также устанавливает свойство *orders* в классе *Users* через аргумент ключевого слова *backref*, которое упорядочивается по *order_id*. Директиве *relationship* требуется целевой класс для отношения, и она может дополнительно включать обратное отношение для целевого класса. *SQLAlchemy* знает, как сопоставить заданный нами *ForeignKey* с классом, который мы определили в отношении. В этом примере команда `ForeignKey(users.user_id)` сопоставляется с классом *User* через атрибут `__tablename__` пользователей и формирует связь. В строке 
```
User=relationship('Users', backref=backref('orders', order_by=order_id))
```
устанавливается связь *один ко многим*.

Также возможно установить взаимно-однозначное отношение *один к одному*. В следующем примере класс *Line_items* имеет взаимно-однозначное отношение с классом *Listings*. Аргумент ключевого слова `uselist = False` определяет его как взаимно однозначное отношение. Здесь используется более простая обратная ссылка, поскольку нам не нужно контролировать порядок.
```
class Line_items(Base):
    
    __tablename__='line_items'
    item_id=Column(Integer(), primary_key=True)
    order_id=Column(Integer(), ForeignKey('orders.order_id'))
    listing_id=Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date=Column(DateTime(), nullable=False, default=datetime.now)
    item_end_date=Column('item_end_date', DateTime(), nullable=False)
    
    Order=relationship("Orders", backref=backref('line_items', order_by=line_item_id))
    Listing=relationship("Listings", uselist=False))
```

<a id=T_3_2></a>
[<= ](#T_3_1)||[ К оглавлению ](#Ref)||[ =>](#T_3_3)
# 3.2. Сеанс

Сеанс - это способ взаимодействия ORM SQLAlchemy с базой данных. Он "обертывает" соединение с базой данных через механизм и предоставляет карту идентификации для объектов, которые вы загружаете через сеанс или связываете с сеансом. Карта идентификации - это структура данных, подобная кешу, которая содержит уникальный список объектов, определяемый таблицей объекта и первичным ключом. Сеанс также "обертывает" транзакцию, и эта транзакция будет открыта до тех пор, пока сеанс не будет зафиксирован или не пройзойдет откат, что очень похоже на процесс, описанный в теме *Core*.

Для нового сеанса SQLAlchemy предоставляет класс *sessionmaker*, чтобы гарантировать, что сеансы могут быть созданы с одинаковыми параметрами во всем приложении. SQLAlchemy делает это путем создания класса сеанса (Session), который настроен в соответствии с аргументами, переданными в класс *sessionmaker*, который следует использовать только один раз в глобальной области действия приложения и рассматривать как параметр конфигурации. Создадим новый сеанс, связанный с базой данных SQLite в памяти:

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker # 1 

#engine = create_engine('sqlite:///:memory:') # 2a
engine = create_engine('sqlite:///Listings.db') # 2b

Session = sessionmaker(bind=engine) # 3

session = Session() # 4

1. Импорт модуля создания сеанса *sessionmaker*.
2. База данных SQLite a) в памяти, b) на диске.
3. Определение класса сеанса с привязкой к механизму.
4. Создание сеанса.

Теперь у нас есть сеанс, который мы можем использовать для взаимодействия с базой данных. Определим классы таблиц базы данных. Дополнительно добавим методы `__repr__`, чтобы упростить просмотр и воссоздание экземпляров объектов.

In [ ]:
from datetime import datetime

from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean, DateTime,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()


class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate = datetime.now)

    def __repr__(self):
        return "User(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)

    
class Order(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    created_on = Column(DateTime(), default = datetime.now)
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)
    
    User=relationship("User", backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Order(user_id='{self.user_id}', " \
                    "user_id='{self.user_id}', " \
                    "created_on='{self.created_on}')".format(self=self)
    

class Line_item(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)
    
    Order=relationship("Order", backref=backref('line_items', order_by=item_id))
    Listing=relationship("Listing", uselist=False)
    
    def __repr__(self):
        return "Line_item(order_id='{self.order_id}', " \
                        "listing_id='{self.listing_id}', " \
                        "item_start_date='{self.item_start_date}', " \
                        "item_end_date='{self.item_end_date}')".format(self=self)


class Host(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Host(host_id='{self.host_name}')".format(self=self)
    
    
class Neighbourhood(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)
    def __repr__(self):
        return "Neighbourhood(neigh_name='{self.neigh_name}')".format(self=self)
    
    
class Room_type(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Room_type(room_type_name='{self.room_type_name}')".format(self=self)

### Задание 3.2.1.

Создайте класс *Property_type* для справочника типов собственности *property_types*

In [ ]:
# Ваш код здесь


In [ ]:
class Property_type(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Property_type(property_type_name='{self.property_type_name}')".format(self=self)

### Задание 3.2.2.
Создайте класс *Listing* с отношениями к классам *Host, Neighbourhood, Room_type, Property_type, Line_item* и методами `__repr__`

In [ ]:
# Ваш код здесь


In [ ]:
class Listing(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        ForeignKeyConstraint(['host_id'], ['hosts.host_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    
    Host=relationship('Host', backref=backref('listings', order_by=listing_id))
    Neighbourhood=relationship('Neighbourhood', backref=backref('listings', order_by=listing_id))
    Property_type=relationship('Property_type', backref=backref('listings', order_by=listing_id))
    Room_type=relationship('Room_type', backref=backref('listings', order_by=listing_id))

    
    def __repr__(self):
        return "Listing(listing_name='{self.listing_name}', " \
                       "listing_url='{self.listing_url}', " \
                       "amenities='{self.amenities}', " \
                       "bedrooms='{self.bedrooms}', " \
                       "price='{self.price}')".format(self=self)

In [ ]:
Base.metadata.create_all(engine)

<a id=T_3_3></a>
[<= ](#T_3_2)||[ К оглавлению ](#Ref)||[ =>](#T_3_4)

# 3.3. Вставка данных

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./Data/ListingsAm.csv', sep=";")
df.head(1)

In [ ]:
df.info()

В наборе имеется несколько строк с пустыми названиями объектов. Включение таких объектов в базу данных запрещено, так как установлено ограничение - запрет на пустое название объекта. Удалим такие строки. Сначала распечатаем их.

In [ ]:
df[df['name'].isna()]

Теперь удалим эти записи

In [ ]:
to_drop_idx = list(df[df['name'].isna()].index)
df.drop(to_drop_idx, axis=0, inplace=True)

Также мы видим, что в части записей информация о bedrooms и beds отсутствует. Установим нулевые значения.

In [ ]:
df['beds'].fillna(0, inplace=True)
df['bedrooms'].fillna(0, inplace=True)

Подготовим справочник районов

In [ ]:
neigh_df = df["neighbourhood_cleansed"].value_counts().sort_index().reset_index()
neigh_df.index = range(1, len(neigh_df)+1)
neigh_dict = neigh_df["index"].to_dict()
neigh_dict

In [ ]:
for key, value in neigh_dict.items():
    w = Neighbourhood(neigh_id = key, neigh_name = value)
    session.add(w)
session.commit()

print(w.neigh_id, w.neigh_name)

### Задание 3.3.1.
Наполнить данными справочники владельцев недвижимости, типов комнат и типов собственности. При создании справочника владельцев в качестве значения первичного ключа указать значение индекса владельца из *ListingsAm.csv*

Чтобы добавить запись, содержащую значения нескольких столбцов, например первичного ключа и названия, можно в одном выражении указать несколько параметров класса таблицы:

```
w=ClassName(Column_1=value_1, Column_2=value_2, ... Column_n=value_n)
sesseion.add(w)
```

In [ ]:
# Ваш код здесь


In [ ]:
H=df[['host_id','host_name']].groupby(["host_id", "host_name"])['host_id'].count()
H.head()

In [ ]:
for item in H.index:
    w = Host(host_id=item[0], host_name=item[1])
    session.add(w)
session.commit()
w.host_id, w.host_name

Проверка, сколько записей в таблице *hosts*.

In [ ]:
from sqlalchemy.sql import select, func
connection=engine.connect()
s=select([func.count(Host.__table__)])
print(str(s))
rp = connection.execute(s)
rp.scalar()

Наполнение справочника типов комнат

In [ ]:
room_type_df = df["room_type"].value_counts().sort_index().reset_index()
room_type_df.index = range(1, len(room_type_df)+1)
room_type_dict=room_type_df["index"].to_dict()

for value in room_type_dict.values():
    w = Room_type(room_type_name = value)
    session.add(w)

session.commit()
w.room_type_id, w.room_type_name

Наполнение справочников типов собственности

In [ ]:
property_type_df = df["property_type"].value_counts().sort_index().reset_index()
property_type_df.index = range(1, len(property_type_df)+1)
property_type_dict = property_type_df["index"].to_dict()

for value in property_type_dict.values():
    w = Property_type(property_type_name = value)
    session.add(w)

session.commit()
w.property_type_id, w.property_type_name

### Задание 3.3.2.

Наполнить данными таблицу *listings*

In [ ]:
# Ваш код здесь

In [ ]:
df.head(1)

В процессе работы выдается предупреждение о том, что тип данных "десятичное число" не поддерживается напрямую. Чтобы отключить такие сообщения, выполним команду
```
import warnings; warnings.simplefilter('ignore')
```

In [ ]:
import warnings; warnings.simplefilter('ignore')

neigh_dict_reversed = {}
for key, value in neigh_dict.items():
    neigh_dict_reversed[value] = key

property_type_dict_reversed = {}
for key, value in property_type_dict.items():
    property_type_dict_reversed[value] = key

room_type_dict_reversed = {}
for key, value in room_type_dict.items():
    room_type_dict_reversed[value] = key
    
for row in df.index[::]:
    w = Listing(
        listing_id = int(df.loc[row, 'id']),
        listing_name = df.loc[row, 'name'],
        listing_url = df.loc[row, 'listing_url'],
        host_id = int(df.loc[row, 'host_id']),
        neighbourhood_id = int(neigh_dict_reversed[df.loc[row, 'neighbourhood_cleansed']]),
        amenities = df.loc[row, 'amenities'],
        property_type_id = int(property_type_dict_reversed[df.loc[row, 'property_type']]),
        room_type_id = int(room_type_dict_reversed[df.loc[row, 'room_type']]),
        bedrooms = int(df.loc[row, 'bedrooms']),
        beds = int(df.loc[row, 'beds']),
        price = int(df.loc[row, 'price'])
    )
    
    session.add(w)

session.commit()
print(w.listing_id, w.listing_url, w.listing_name, 
      w.host_id, w.neighbourhood_id, w.property_type_id, w.room_type_id)

## Наполнение таблиц *users, orders, line_items*

Таблица *users*

In [ ]:
User.__table__

In [ ]:
user_list=[(1,'Nicolas','nicolas@rambler.ru','+7-929-616-88-77','@#$%890'),
           (2,'Lida','lidaok@gmail.com','+7-929-616-88-77','yyT$%333'),
           (3,'Vera','lveramuns@gmail.com','+7-353-214-12-90','yyT$%333'),
           (4,'Ivan','ivaturgenev@yandex.ru','+7-047-121-89-95','tT6^7&#20Oy'),
           (5,'Svetlana','svetaivanova@microsoft.com','+7-812-555-48-71','SD%@OUsdc7')
          ]

for item in user_list:
    w = User(
        user_id=item[0],
        username=item[1],
        email_address=item[2],
        phone=item[3],
        password=item[4],
        created_on=datetime.now()
    )
    session.add(w)
session.commit()

print(w.user_id, w.username, w.email_address, 
      w.phone, w.password, w.created_on, w.updated_on)

Таблица *orders*

In [ ]:
Order.__table__

In [ ]:
order_list = [(1,1,'2020-05-01'),(2,1,'2020-05-02'),
              (3,2,'2020-05-03'),(4,3,'2020-05-04'),
              (5,3,'2020-05-05'),(6,4,'2020-05-06'),
              (7,4,'2020-05-07'),(8,4,'2020-05-08'),
              (9,5,'2020-05-09'),(10,5,'2020-05-10'),
              (11,5,'2021-05-09'),(12,5,'2021-05-09')]

for item in order_list:
    w = Order(
        order_id=item[0],
        user_id=item[1],
        created_on=datetime.strptime(item[2], '%Y-%m-%d')
    )
    session.add(w)
session.commit()

print(w.order_id, w.user_id, w.created_on)

Таблица *line_items*

In [ ]:
Line_item.__table__

In [ ]:
line_list=[(1,1,20168,'2020-05-07','2020-05-17'),
           (2,1,27886,'2020-05-17','2020-05-27'),
           (3,2,28871,'2020-08-01','2020-08-10'),
           (4,3,29051,'2020-02-06','2020-03-17'),
           (5,4,2550571,'2020-04-07','2020-06-01'),
           (6,4,18225467,'2020-05-07','2020-06-01'),
           (7,5,3908795,'2020-07-07','2020-08-19'),
           (8,6,16550704,'2020-08-03','2020-08-11'),
           (9,7,3908795,'2020-09-02','2020-09-12'),
          ]

for jtem in line_list:
    w = Line_item(
        item_id=jtem[0],
        order_id=jtem[1],
        listing_id=jtem[2],
        item_start_date=datetime.strptime(jtem[3],'%Y-%m-%d'),
        item_end_date=datetime.strptime(jtem[4],'%Y-%m-%d')
    )
    session.add(w)
session.commit()

print(w.item_id, w.order_id, w.listing_id, w.item_start_date, w.item_end_date)

<img src="../Img/Label_02.png">

Семинар

30 апреля 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

30 апреля 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

15 мая 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

<a id=T_3_4></a>
[<= ](#T_3_3)||[ К оглавлению ](#Ref)||[ =>](#T_3_5)
# 3.4. Запросы

### Методы *first(), all(), one().* Ограничение *limit()*.

In [ ]:
session.query(Room_type).all()

In [ ]:
session.query(Property_type).limit(3).all()

In [ ]:
session.query(Neighbourhood.neigh_id, Neighbourhood.neigh_name).first()

In [ ]:
session.query(Neighbourhood.neigh_id, Neighbourhood.neigh_name).limit(7).all()

In [ ]:
for item in session.query(Room_type):
    print(item.room_type_name, item.room_type_id)

In [ ]:
session.query(Listing).limit(1).one()

In [ ]:
session.query(Listing.listing_id, Listing.neighbourhood_id, Listing.host_id).limit(3).all()

In [ ]:
session.query(Host.host_id, Host).limit(3).all()

Другие методы: *scalar()*

## 3.4.1. Ограничение числа строк запроса

В предыдущих примерах мы использовали метод *first()*, чтобы вернуть только одну строку. Хотя наш *query()* дал нам одну запрошенную строку, фактический запрос прошел и получил доступ ко всем результатам, а не только к отдельной записи. Если мы хотим ограничить запрос, мы можем использовать нотацию среза массива вместо оператора *limit*.

In [ ]:
session.query(Listing.listing_id, 
              Listing.room_type_id,
              Listing.property_type_id,
              Listing.neighbourhood_id,
              Listing.host_id
             )[:4]

## 3.4.2. Упорядочивание

Упорядочивание по возрастанию

In [ ]:
session.query(Listing.listing_id, 
              Listing.listing_name,
              Listing.neighbourhood_id
             ).order_by(Listing.listing_name)[:4]

Упорядочивание по убыванию

In [ ]:
for item in session.query(Listing.listing_id, 
                  Listing.listing_name,
                  Listing.neighbourhood_id
                 ).order_by(Listing.listing_name.desc())[:4]:
    print('{} - {}'.format(item.listing_name, item.neighbourhood_id))

## 3.4.3. Встроенные функции

SQLAlchemy также может использовать встроенные функции SQL, поддерживаемые базой данных. Две наиболее часто используемые функции - это *sum()* и *count()*. Чтобы использовать эти функции нужно импортировать модуль *sqlalchemy.func*.

In [ ]:
from sqlalchemy import func
rec_sum=session.query(func.sum(Listing.price)).scalar()
print(rec_sum)

Метод *scalar()* возвращает значение крайнего левого столбца первой строки. В предыдущем примере результатом является одно число. Если использовать метод *first()* для аналогичного запроса, то результатом станет кортеж.

In [ ]:
rec_count=session.query(func.count(Listing.listing_id)).first()
print(rec_count)

Результат функции содержится в столбце с автоматически сформированным именем, таким как *count_1*. Чтобы присовить столбцу результата другое имя и использовать его в дальнейшем, испольльзуем функцию *label().*

In [ ]:
summa=session.query(func.sum(Listing.price).label('overall_price')).first()
print(summa)
print(summa.keys())
print(summa.overall_price)

## 3.4.4. Фильтрация

Фильтрация осуществляется посредством "пристегивания" к запросу выражения фильтрации с помощью метода *filter()*. Обычно такое выражение содержит столбец, оператор и значение или другой столбец. Можно прикреплять к запросу несколько фильтров или перечислять условия фильтрации через запятую в одном фильтре. Для фильтрации используют методы *filter()* и *filter_by*.

## Метод *filter()*

In [ ]:
from pprint import pprint
record=session.query(Listing.listing_id, Listing).filter(Listing.bedrooms==6).all()
pprint(record)

In [ ]:
record=session.query(Listing.listing_id, Listing) \
    .filter(Listing.bedrooms==6) \
    .filter(Listing.price>600) \
    .all()
pprint(record)

## Метод *filter_by()*

Метод *filter_by()* использует несколько иной синтаксис. Имя класса не используется, вместо этого используется атрибут главного класса запроса, либо класса, который последним был присоединен к запросу. Также вместо оператора сравнения этот метод использует оператор присваивания.

```
record=session.query(Listing.listing_id, Listing) \
    .filter_by(bedrooms=4, price=679).all()
print(record)
```

## Условные методы

In [ ]:
import pandas as pd
CM=pd.read_csv('./Data/ClauseMethods.csv', sep=';')
CM

In [ ]:
session.query(Host.host_id, Host.host_name) \
    .filter(Host.host_name.like('%Alexander%')).all()

## Логические связки

### *and_(), or_(), not_()*

In [ ]:
from sqlalchemy import and_, or_, not_

session.query(Listing.listing_id, Listing).filter(
    and_(
        Listing.bedrooms>6,
        Listing.price.between(600, 700)
    )
).all()

## 3.4.5. Обновление. Метод *update()*

Рассмотрим два способа обновления данных. Найдем в базе данных объект размещения с ценой 679.

In [ ]:
session.query(Listing.listing_id, Listing).filter(Listing.price==679).all()

Увеличим цену на 1.

In [ ]:
q = session.query(Listing)
w = q.filter(Listing.listing_id == 1237639).one()
print(w)
w.price = w.price + 1
session.commit()

print(w.price)

И еще на 1.

In [ ]:
q = session.query(Listing)
q = q.filter(Listing.listing_id == 1237639)
q.update({Listing.price: Listing.price + 1})

print(q.first())

## 3.4.6.Удаление. Метод *delete()*

Рассмотрим способы удаления данных. Найдём все объекты, в названии которых имеется фраза "bright & quiet".

In [ ]:
filter_condition=Listing.listing_name.like('%bright & quiet%')

session.query(Listing.listing_id, Listing.listing_name) \
    .order_by(Listing.listing_id) \
    .filter(filter_condition).all()

Удалим все такие записи

In [ ]:
# 1
q = session.query(Listing)
q = q.filter(filter_condition)
records_to_del = q.all()
for record in records_to_del:
    session.delete(record)
session.commit()

Проверим, остались ли записи, соответствующие условию

In [ ]:
q=session.query(Listing)
q=q.filter(Listing.listing_name.like(filter_condition))
q.all()

## 3.4.7. Соединение таблиц. Методы *join* и *outerjoin*

Найдем все объекты размещения, заказанные пользователем *Vera*. Как мог бы выглядеть поиск этих объектов без *join?* Например, так:

In [ ]:
q=session.query(Listing.listing_id, Listing.listing_name)
q=q.filter(Listing.listing_id==Line_item.listing_id)
q=q.filter(Line_item.order_id==Order.order_id)
q=q.filter(Order.user_id==User.user_id)
q=q.filter(User.username=='Vera')

q.all()

Или так:

In [ ]:
q4=session.query(User.user_id).filter(User.username=='Vera')
q3=session.query(Order.order_id).filter(Order.user_id==q4)
q2=session.query(Line_item.listing_id).filter(Line_item.order_id.in_(q3))
q1=session.query(Listing.listing_id, Listing.listing_name)
q1=q1.filter(Listing.listing_id.in_(q2))

q1.all()

Выполним этот запрос с использованием соединения *join*

In [ ]:
q=session.query(Listing.listing_id, Listing.listing_name)
q=q.join(Line_item).join(Order).join(User)
q=q.filter(User.username=='Vera')

q.all()

## 3.4.8. Группировка. Метод *group_by*

Найдём, сколько объектов заказали клиенты.

In [ ]:
q=session.query(User.username, func.count(Listing.listing_id).label('cnt'))
q=q.join(Order).join(Line_item).join(Listing)
q=q.group_by(User.user_id)
print(str(q))
q.all()

В нашей  базе данных есть пользователь Svetlana с user_id=5, который разместил четыре заказа:

In [ ]:
sv_orders=session.query(Order.order_id).filter(Order.user_id==5)
sv_orders.all()

при этом не сделал ни одной детализации, т.е. не указал конкрентых объектов:

In [ ]:
sv_items=session.query(func.count(Line_item.listing_id))
sv_items=sv_items.filter(Line_item.order_id.in_(sv_orders))
sv_items.scalar()

С помощью *outerjoin* распечатаем число объектов для каждого клиента, включая клиентов, которые не детализировали ни однго заказа.

In [ ]:
q=session.query(User.username, func.count(Listing.listing_id).label('cnt'))
q=q.join(Order).outerjoin(Line_item).outerjoin(Listing)
q=q.group_by(User.user_id)
print(str(q))
q.all()

## 3.4.9. Прямые запросы

In [ ]:
from sqlalchemy import text

In [ ]:
q=session.query(User).filter(text("username=='Lida'"))
print(q.all())

## Выполните задания

С помощью SQLAlchemy ORM выполните задания

1. Создать одного пользователя, для него один заказ, в этом заказе детализировать два объекта в Амстердаме на майские праздники с 1 по 11 мая.
2. Распечатать число объектов в каждом районе.
3. Найти минимальную и максимальную цену объекта в каждом районе.
4. Найти все объекты в районе *Osdorp* с ценой выше средней по этому району.
5. Комнат какого типа больше всего заказано пользователями? 

<img src="../Img/Label_02.png">

Семинар

14 мая 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

14 мая 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

22 мая 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

<a id=T_3_5></a>
[<= ](#T_3_4)||[ К оглавлению ](#Ref)||[ =>](#T_3_6)

# 3.5. Отображение

Чтобы отобразить базу данных, вместо класса *declarative_base* будем использовать базовый класс автосопоставления *automap_base*. Начнем с создания базового объекта *Base.*

In [ ]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

Теперь нам необходим механизм соединения с базой данных, которую мы хотим отобразить

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Data/Students_2021.sqlite')

С базовым объектом и настройкой движка у нас есть все необходимое для отображения базы данных. Метода *prepare* объекта *Base* просканирует все, что доступно в движке, который мы только что создали, и сделает отображение.

In [ ]:
Base.prepare(engine, reflect=True)

Эта строка кода - все, что нужно, чтобы отобразить всю базу данных! Теперь объекты ORM для каждой таблицы доступны в свойстве *classes* объекта *Base*.

In [ ]:
Base.classes.keys()

Если к отображенной базе данных нужно добавить таблицу. Добавление таблицы вручную. Пример.
```
from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from datetime import datetime
from sqlalchemy import DateTime

# Инструкцию Base = declarative_base() не используем.
# Вместо этого используем созданный выше Base = automap_base(). 
# Новый Base не создаем, так как нам надо добавить таблицу 
# не в новую, а в уже отображенную БД

class Subj_lect(Base):
    __tablename__ = 'subj_lect'

    subj_lect_id = Column(Integer(), primary_key = True)
    lecturer_id = Column(Integer())
    subj_id = Column(Integer())
    
    __table_args__ = (
        ForeignKeyConstraint(['lecturer_id'],['lecturer.lecturer_id']),
        ForeignKeyConstraint(['subj_id'], ['subject.subj_id']),
    )

Base.metadata.create_all(engine)
```

Создадим объекты для ссылок на таблицы

In [ ]:
city,exam_marks,student,university,subject,lecturer,subj_lect = Base.classes.values()
#Album,Artist,Customer,Employee,Genre,Invoice,InvoiceLine,Track,MediaType,Playlist = Base.classes.values()

Выполним запрос. Распечатаем информацию о предметах

In [ ]:
from sqlalchemy.orm import Session
session = Session(engine)

In [ ]:
for record in session.query(subject).all():
    print(record.subj_id, record.subj_name, record.hour)

Распечатаем имена, фамилии преподавателей и названия предметов, которые они ведут. Для каждого использования метода *join* уточним атрибуты соединения.

In [ ]:
q=session.query(lecturer.name, lecturer.surname, subject.subj_name)
q=q.join(subj_lect, lecturer.lecturer_id==subj_lect.lecturer_id)
q=q.join(subject, subj_lect.subj_id==subject.subj_id)
q.all()

Автосопоставление *Automap* может автоматически отображать и устанавливать отношения «много к одному», «один ко многим» и «много ко многим» посредством создания свойства `<related_object>_collection`.

In [ ]:
lecturer.__table__

In [ ]:
university.__table__

In [ ]:
u = session.query(university).filter(university.univ_id==46).one()
for l in u.lecturer_collection:
    print(u.univ_name, l.name, l.surname)

Дополнительно об отображении данных в <a href="https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html#specifying-classes-explcitly">документации</a>

<a id=T_3_6></a>
[<= ](#T_3_5)||[ К оглавлению ](#Ref)||[ =>](#T_3_7)

# 3.6. Сеанс более подробно

Когда мы используем запрос для получения объекта, мы возвращаем объект, связанный с сеансом. Этот объект может перемещаться через несколько состояний по отношению к сеансу. Существует четыре возможных состояния экземпляров объекта данных:
- Transient (временное): экземпляр не находится в сеансе и отсутствует в базе данных.
- Pending (в ожидании): экземпляр был добавлен в сеанс с помощью *add()*, но не был сброшен или зафиксирован.
- Persistent (постоянное): объект в сеансе имеет соответствующую запись в базе данных.
- Detached (отсоединенное): экземпляр больше не подключен к сеансу, но имеет запись в базе данных.

Мы можем наблюдать, как экземпляр проходит свои состояния. Выполним отображение *Listings.db*

In [ ]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from datetime import datetime
engine = create_engine('sqlite:///Listings.db')
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [ ]:
Base.classes.keys()

In [ ]:
listings = Base.classes.listings
line_items = Base.classes.line_items
orders = Base.classes.orders
users = Base.classes.users

Создадим экземпляр детализации заказа пользователя Svetlana. Вспомним сначала, какие заказы она разместила.

In [ ]:
sv_orders = session.query(orders).filter(orders.user_id==5).all()
for item in sv_orders:
    print(item.order_id, item.created_on)

Создадим детализацию заказа № 9. Пусть это будет объект 29051 Amsterdam Center Entire Apartment в период с 1 по 11 мая 2021 года.

In [ ]:
a_line_item = line_items(order_id = int(9),
                         listing_id = int(29051),
                         item_start_date = datetime.strptime('2021-05-01','%Y-%m-%d'),
                         item_end_date = datetime(2021,5,11)
                        )

Чтобы увидеть состояние экземпляра, применим метод SQLAlchemy *inspect()*.

In [ ]:
from sqlalchemy import inspect
insp = inspect(a_line_item)

Теперь мы можем обратиться к свойствам объекта *insp: transient, pending, persistent, detached*.

In [ ]:
print(insp.transient, insp.pending,)

Пройдем в цикле по всем свойствам.

In [ ]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

Как видим в выходных данных, текущее состояние нашего экземпляра детализации заказов является временным, то есть в состоянии, в котором находятся вновь созданные объекты до того, как они будут сброшены или зафиксированы в базе данных. Если мы добавим *a_line_item* в текущий сеанс и повторно запросим состояние, мы получим следующий результат:

In [ ]:
session.add(a_line_item)

In [ ]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

Состояние изменилось. Подтвердим изменения.

In [ ]:
session.commit()

In [ ]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

Для демонстрации отсоединенного состояния выполним метод *expunge,* который используется для удаления объекта из сессии.

In [ ]:
session.expunge(a_line_item)

In [ ]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

Посмотрим теперь, как использовать  инспектор для отображения истории изменений. Вернём объект в сессию и изменим номер объекта. Пусть теперь это объект 41125 Amsterdam Center Entire Apartment.

In [ ]:
session.add(a_line_item)
a_line_item.listing_id = 41125

Применим свойство инспектора *modified*

In [ ]:
insp.modified

Мы получили значение истины. Используем теперь коллекцию *attr*, чтобы узнать, что было изменено.

In [ ]:
for attr, attr_state in insp.attrs.items():
    if attr_state.history.has_changes(): # Проверка состояния атрибута
        print('{}: {}'.format(attr, attr_state.value))
        print('History: {}\n'.format(attr_state.history)) # История измененного объекта

В результате мы получили данные об изменении идентификатора объекта. На этом примере мы изучили, как объекты взаимодействуют с сеансом.

<img src="./Img/Label_02.png">

Семинар

21 мая 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

28 мая 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

29 мая 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

<a id=T_3_7></a>
[<= ](#T_3_6)||[ К оглавлению ](#Ref)||[ =>](#T_3_8)

# 3.7. Исключения

В SQLAlchemy может возникать множество исключений, рассмотрим исключение *MultipleResultsFound*  Научившись обрабатывать это исключения, мы будем лучше подготовлены к работе с другими исключениями.

### Исключение *MultipleResultsFound*
Это исключение возникает, когда мы используем метод запроса *.one(),* но получаем более одного
результата обратно. Прежде чем начать работу по обработке исключений, сделаем отображение базы данных *Listing.db*

In [ ]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from datetime import datetime
engine = create_engine('sqlite:///Listings.db')
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)
hosts,line_items,orders,users,listings,neighbourhoods,property_types,room_types=Base.classes.values()

Отберем все заказы и применим метод *one()*. Выполнение такого кода приводит к ошибке:
```
result = session.query(orders).one()

---------------------------------------------------------------------------
MultipleResultsFound                      Traceback (most recent call last)
~\anaconda3\lib\site-packages\sqlalchemy\orm\query.py in one(self)
   3346         try:
-> 3347             ret = self.one_or_none()
   3348         except orm_exc.MultipleResultsFound:

~\anaconda3\lib\site-packages\sqlalchemy\orm\query.py in one_or_none(self)
   3324             raise orm_exc.MultipleResultsFound(
-> 3325                 "Multiple rows were found for one_or_none()"
   3326             )

MultipleResultsFound: Multiple rows were found for one_or_none()

During handling of the above exception, another exception occurred:

MultipleResultsFound                      Traceback (most recent call last)
<ipython-input-14-80567daed371> in <module>
----> 1 result = session.query(orders).one()

~\anaconda3\lib\site-packages\sqlalchemy\orm\query.py in one(self)
   3348         except orm_exc.MultipleResultsFound:
   3349             raise orm_exc.MultipleResultsFound(
-> 3350                 "Multiple rows were found for one()"
   3351             )
   3352         else:

MultipleResultsFound: Multiple rows were found for one()
```

Ошибка произошла из-за того, что более одной записи отвечают условиям отбора.
- Тип ошибки `MultipleResultsFound`
- Строка ошибки `----> 1 result = session.query(orders).one()`
- Пояснение `-> 3350                 "Multiple rows were found for one()"`

Аналогичная ошибка `NoResultFound` может возникнуть, если используется метод *one()* и не отобрано ни одной записи:

```
result=session.query(orders).filter(orders.order_id==100).one()

---------------------------------------------------------------------------
NoResultFound                             Traceback (most recent call last)
<ipython-input-17-53f5e4a13e4c> in <module>
----> 1 result=session.query(orders).filter(orders.order_id==100).one()

~\anaconda3\lib\site-packages\sqlalchemy\orm\query.py in one(self)
   3352         else:
   3353             if ret is None:
-> 3354                 raise orm_exc.NoResultFound("No row was found for one()")
   3355             return ret
   3356 

NoResultFound: No row was found for one()
```

Если мы хотим обработать это исключение, чтобы наша программа не прекращала выполнение и выводила более полезное сообщение об исключении, используем блок Python try / except.

In [ ]:
from sqlalchemy.orm.exc import MultipleResultsFound

try:
    result = session.query(orders).one()
except MultipleResultsFound as error:
    print('У нас больше одного заказа')

In [ ]:
from sqlalchemy.orm.exc import NoResultFound
try:
    result = session.query(orders).filter(orders.order_id==150)
    result = result.one()
except NoResultFound as error:
    print('Не найдено ни одного результата')

<a id=T_3_8></a>
[<= ](#T_3_7)||[ К оглавлению ](#Ref)||
# Задания

Выполните задания, используя методы SQL Alchemy ORM

### I. Оператор *select*, упорядочивание, агрегирующие функции, неопределнные значения, групповые функции, условные операторы
1. Создайте отображение базы данных студентов *Students_2021.sqlite*
2. Напишите запрос, позволяющий получить из таблицы *exam_marks* значения столбца *mark* (экзаменационная оценка) для всех студентов, исключив из списка повторение одинаковых строк. Результат не должен содержать значений (None). Упорядочить результат по возрастанию значения оценки.
3. Напишите запрос для получения списка студентов без определенного места жительства. Результат должен содержать идентификатор студента, фамилию, имя.
4. Напишите запрос для получения списка студентов, проживающих в Воронеже и не получающих стипендию.
5. Напишите запрос для получения списка университетов, расположенных в Москве и имеющих рейтинг меньший, чем у НГУ. Значение рейтинга НГУ получите с помощью отдельного запроса или подзапроса.
6. Напишите запрос, выполняющий вывод находящихся в таблице EXAM_MARKS номеров предметов обучения, экзамены по которым сдавались между 1 и 21 марта 2020 г.
7. Напишите запрос, который выполняет вывод названий предметов обучения, начинающихся на букву ‘И’.
8. Напишите запрос, возвращающий фамилии и имена студентов, у которых имена начинаются на букву ‘И’ или ‘С’.
9. Напишите запрос для получения списка предметов обучения, названия которых состоят из более одного слова.
10. Напишите запрос для получения списка студентов, фамилии которых состоят из трех букв.
11. Составьте запрос для таблицы STUDENT таким образом, чтобы получить результат в следующем виде. Распечатайте первые 9 записей результата.
```
И. Иванов 	 1982-12-03
П. Петров 	 1980-12-01
В. Сидоров 	1979-06-07
...
```
12. Напишите запрос для получения списка студентов, фамилии которых начинаются на ‘Ков’ или на ‘Куз’.
13. Напишите запрос для получения списка предметов, названия которых оканчиваются на ‘ия’.
14. Составьте запрос, выводящий фамилии, имена студентов и величину получаемых ими стипендий, при этом значения стипендий должны быть увеличены в 100 раз. Распечатайте первые 10 результатов.
15. Составьте запрос для таблицы UNIVERSITY таким образом, чтобы выходная таблица содержала всего один столбец в следующем виде: Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.
16. Напишите запрос для подсчета количества студентов, сдававших экзамен по предмету обучения с идентификатором 10.
17. Напишите запрос, который позволяет подсчитать в таблице EXAM_MARKS количество различных предметов обучения.
18. Напишите запрос, который для каждого студента выполняет выборку его идентификатора и минимальной из полученных им оценок.
19. Напишите запрос, который для каждого конкретного дня сдачи экзамена выводит данные о количестве студентов, сдававших экзамен в этот день.
20. Напишите запрос, выдающий идентификатор студента и его средний балл. 
21. Напишите запрос, выдающий средний балл для каждого экзамена.
22. Напишите запрос, определяющий количество сдававших студентов для каждого предмета, по которому был экзамен.
23. Напишите запрос для определения количества предметов, изучаемых на каждом курсе.
24. Для каждого университета напишите запрос, выводящий суммарную стипендию обучающихся в нем студентов, с последующей сортировкой списка по этому значению.
25. Для каждого студента напишите запрос, выводящий идентификатор студента и среднее значение оценок, полученных им на всех экзаменах.
26. Напишите запрос, выводящий количество студентов, проживающих в каждом городе. Список отсортировать в порядке убывания количества студентов.